In [1]:
import boto3
import csv
import json
import time

In [2]:
region = 'us-east-2'
streamName = 'blossom-data-eng-emily'

In [3]:
#inititalized kinesis client
kinesis = boto3.client('kinesis',region_name = 'us-east-2')

#list all active streams
kinesis.list_streams()

{'StreamNames': ['blossom-data-eng-Priscilla',
  'blossom-data-eng-clifford',
  'blossom-data-eng-dennis',
  'blossom-data-eng-emily',
  'blossom-data-eng-giftysdovie',
  'blossom-data-eng-jedidiah',
  'blossom-data-eng-justice-akwensi',
  'blossom-data-eng-michael_asante',
  'blossom-data-eng-richmond',
  'blossom-data-eng-username',
  'blossom-eng-Priscilla',
  'blossom-eng-juliuslotsu',
  'blossom-eng-michaelasante',
  'blossom-eng-omaj',
  'kojo-kinesis-test',
  'real_estate'],
 'HasMoreStreams': False,
 'ResponseMetadata': {'RequestId': 'd2423287-6e4a-d193-8415-b85e8df148d6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd2423287-6e4a-d193-8415-b85e8df148d6',
   'x-amz-id-2': '8jhqIbCnElcs7XLwjWUL5GHdrnq+XMrmDifOtBtqlVHc6rLklvcdU7RUX+4IpBpJykw2n+WzZMSGscPiIxCXQdQKSYtSEuiw',
   'date': 'Tue, 26 Nov 2019 20:18:33 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '463'},
  'RetryAttempts': 0}}

In [4]:
stream_description = kinesis.describe_stream(StreamName = streamName)
stream_description.keys()
stream_description['StreamDescription']['Shards']
stream_description['StreamDescription']['Shards'][0]


{'ShardId': 'shardId-000000000002',
 'ParentShardId': 'shardId-000000000000',
 'AdjacentParentShardId': 'shardId-000000000001',
 'HashKeyRange': {'StartingHashKey': '0',
  'EndingHashKey': '340282366920938463463374607431768211455'},
 'SequenceNumberRange': {'StartingSequenceNumber': '49601664250058746018402576470843268827267427573564440610'}}

In [5]:
stream_description['StreamDescription']['Shards']

[{'ShardId': 'shardId-000000000002',
  'ParentShardId': 'shardId-000000000000',
  'AdjacentParentShardId': 'shardId-000000000001',
  'HashKeyRange': {'StartingHashKey': '0',
   'EndingHashKey': '340282366920938463463374607431768211455'},
  'SequenceNumberRange': {'StartingSequenceNumber': '49601664250058746018402576470843268827267427573564440610'}}]

CREATING A PRODUCER


In [6]:
from datetime import datetime
import time
import boto3
from looper_scraper import scrape_looper


In [7]:
def get_data(criteria):
    data = scrape_looper(1, add_gps=True)  # ignore the argument 1
    data = data.head(2) # our assumption; first 2 listings are new
    
    # we may have some criteria to filter our data on eg "address='adenta'"
    if criteria:
        data = data.query(criteria)
    
    # for this exercise, we'll only select these columns
#     data = data[['description', 'category','beds', 'baths','price','url','area(mxm)','broker','listing_time',
#                  'amenities','lat','lon','point_of_interest','point_of_interestcategory','distance_to_point_of_interest']]
    
    return data

In [8]:
def listings_producer(stream_name, data):
        response = kinesis.put_record(
            StreamName=stream_name,
            Data=data,
            PartitionKey='blossom',
        )
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            print(f"Records pushed to {stream_name} within kinesis")
        else:
            print("Records not failed to be pushed to kinesis")


We’ll run the scraping and pushing every 30 seconds

In [ ]:
if __name__ == '__main__':
    try:
        while True:
            data = get_data(None)
            data = data.to_json()
            listings_producer(streamName, data)
            time.sleep(10)
            
    except Exception as e:
        print(f"Writing failed. Exiting gracefully due to {e}")

Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
Records pushed to blossom-data-eng-emily within kinesis
Getting page 1 from looperghana
